# THE PANDEMIC'S WORKING PARENTS

#### The Project

The purpose of this project is to shed light on the challenges that working parents are facing during the 2020-2021 COVID-19 pandemic. My analysis of U.S. Census Household Pulse Survey data reveals that in 2020 California’s
job market lost the most parents with schoolchildren in the country, trailing only Nevada and
Michigan. In 2020, California households with PreK-12 children were significantly more likely to lose employment income than parents without children. 


#### The Data
I downloaded the data for weeks 1 through 25 of the pandemic from the U.S. Census Bureau Household Pulse Survey Public Use files (https://www.census.gov/programs-surveys/household-pulse-survey/datasets.html). Each week's data is published in a separate csv file. The Census also publishes weekly data dictionaries in excel format. 

## Importing tools

In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib as plt
import tabula

pd.options.mode.chained_assignment = None # None|'warn'|'raise'
pd.set_option('display.float_format', '{:.2f}'.format)

class color:
   BOLD = '\033[1m'
   END = '\033[0m'

<hr>

## Importing the data

I create an empty list to store the Census Pulse Survey files I downloaded. I then create a loop which imports each file and adds it to my list. 

In [2]:
# creating empty list to store imported dfs
pulse_file_lst = []

# looping through, importing and adding dfs in folder to file_lst
for file_name in os.listdir('pulse_files')[:-1]:
    path = '/Users/carolineghisolfi/Desktop/winter_2021/dataj_pulse/pulse_files/' + file_name
    file = pd.read_csv(path, 
                       dtype={
                           'WEEK': int
                       })
    pulse_file_lst.append(file)

In [3]:
print('\n', color.BOLD + 'First week' + color.END, '\n')
display(pulse_file_lst[0].head(3))
display(pulse_file_lst[0].info())


 First week 



,SCRAM,WEEK,EST_ST,EST_MSA,REGION,HWEIGHT,PWEIGHT,TBIRTH_YEAR,ABIRTH_YEAR,EGENDER,...,PSWHYCHG1,PSWHYCHG2,PSWHYCHG3,PSWHYCHG4,PSWHYCHG5,PSWHYCHG6,PSWHYCHG7,PSWHYCHG8,PSWHYCHG9,INCOME
0,V220000001S10011352410113,22,1,NaN,2,1170.79,3285.40,1978,2,2,...,-88,-88,-88,-88,-88,-88,-88,-88,-88,-88
1,V220000001S10011554410113,22,1,NaN,2,899.97,1683.63,1947,2,2,...,-99,-99,-99,1,-99,1,-99,-99,-99,4
2,V220000001S15010024400123,22,1,NaN,2,2077.84,3887.14,1989,2,1,...,-88,-88,-88,-88,-88,-88,-88,-88,-88,5


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68348 entries, 0 to 68347
Columns: 204 entries, SCRAM to INCOME
dtypes: float64(4), int64(199), object(1)
memory usage: 106.4+ MB


None

I create a master <b>pulse</b> dataframe by concatenating all the files in my list. 

In [4]:
# concatenating dfs in master
pulse = pd.concat(pulse_file_lst)

print('\n', color.BOLD + 'Weeks 1 through 25' + color.END, '\n')
display(pulse.head(3))
display(pulse.info())

print('\n', color.BOLD + 'Weeks in dataframe:' + color.END, np.sort(pulse.WEEK.unique()))


 Weeks 1 through 25 



,SCRAM,WEEK,EST_ST,EST_MSA,REGION,HWEIGHT,PWEIGHT,TBIRTH_YEAR,ABIRTH_YEAR,EGENDER,...,SNAPMNTH4,SNAPMNTH5,SNAPMNTH6,SNAPMNTH7,SNAPMNTH8,SNAPMNTH9,SNAPMNTH10,SNAPMNTH11,SNAPMNTH12,TBEDROOMS
0,V220000001S10011352410113,22,1,NaN,2.00,1170.79,3285.40,1978,2,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,V220000001S10011554410113,22,1,NaN,2.00,899.97,1683.63,1947,2,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,V220000001S15010024400123,22,1,NaN,2.00,2077.84,3887.14,1989,2,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


<class 'pandas.core.frame.DataFrame'>
Int64Index: 2167927 entries, 0 to 108061
Columns: 226 entries, SCRAM to TBEDROOMS
dtypes: float64(153), int64(72), object(1)
memory usage: 3.7+ GB


None


 Weeks in dataframe: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25]


In the process of cleaning this database, I remove rows with missing answers to questions important to my analysis. I record the length of the original database now so I can compare it as I go. 

In [5]:
original_length = len(pulse)
print('\n', color.BOLD + 'Original row count:' + color.END, original_length)


 Original row count: 2167927


<hr>

# Adjusting Weights

The Pulse Survey underwent small but significant changes between phase 1, and phases 2 and 3 (an overview of these changes can be found <a href='https://www.huduser.gov/portal/pdredge/pdr-edge-frm-asst-sec-092820.html'>here</a>). For example, while the survey was conducted on a weekly basis in the first phase, it became biweekly in the second and third. 

Most importantly to this analysis, the Census introduced a new household weight variable in phases 2 and 3. While in phase 1 (weeks 1-12) household weights were provided in  separate weekly excel sheets, in phases 2 and 3 the Census included household weights in the main database's <b>HWEIGHT</b> column. 

In [6]:
pulse[['HWEIGHT', 'WEEK']].groupby(by='WEEK')['HWEIGHT'].min()

WEEK
1      NaN
2      NaN
3      NaN
4      NaN
5      NaN
6      NaN
7      NaN
8      NaN
9      NaN
10     NaN
11     NaN
12     NaN
13   19.99
14   16.30
15   22.25
16    9.02
17   20.99
18   25.55
19   31.13
20   17.11
21   32.51
22   31.62
23   22.93
24   20.27
25   32.90
Name: HWEIGHT, dtype: float64

Once again, I create an empty list and create a loop which stores the weight files in the list. Then, I concatenate the weight files in a master dataframe named <b>weights</b>.

In [7]:
# creating empty list to store imported dfs
weight_file_lst = []

# looping through, importing and adding dfs in folder to file_lst
for file_name in os.listdir('weight_files')[:-1]:
    path = '/Users/carolineghisolfi/Desktop/winter_2021/dataj_pulse/weight_files/' + file_name
    file = pd.read_csv(path)
    weight_file_lst.append(file)
    
# concatenating dfs in master
weights = pd.concat(weight_file_lst)

In [8]:
print('\n', color.BOLD + 'Phase 1 weights' + color.END, '\n')
display(weights.head(3))


 Phase 1 weights 



,WEEK,SCRAM,HWEIGHT,HWEIGHT1,HWEIGHT2,HWEIGHT3,HWEIGHT4,HWEIGHT5,HWEIGHT6,HWEIGHT7,...,HWEIGHT71,HWEIGHT72,HWEIGHT73,HWEIGHT74,HWEIGHT75,HWEIGHT76,HWEIGHT77,HWEIGHT78,HWEIGHT79,HWEIGHT80
0,1,V010000001S10011099370111,1074.75,1074.75,1133.46,832.28,2107.00,1110.27,290.29,1861.68,...,914.35,1167.75,328.65,1024.18,1707.99,992.20,1227.19,1658.08,934.61,1263.89
1,1,V010000001S10011900470112,2147.82,2147.82,3672.59,565.94,2235.81,3380.11,2198.79,2147.58,...,3117.82,2363.21,574.87,3596.52,597.45,2264.79,2067.21,2334.59,1993.23,2214.06
2,1,V010000001S18010744940111,842.56,842.56,1578.21,223.67,927.74,1864.36,612.33,855.89,...,1433.85,746.41,279.66,1126.62,295.25,915.62,701.33,1091.08,716.94,841.38


I am only interested in the main weight, <b>HWEIGHT</b>, so I exclude other calculated weights. 

In [9]:
weights = weights[['WEEK', 'SCRAM', 'HWEIGHT']]

I merge the pulse and weights dataframes. The merge creates two <b>HWEIGHT</b> columns differentiated by the x and y variables. I combine the two in a new <b>HWEIGHT</b> column. 

In [10]:
# Merging dfs
pulse = pd.merge(pulse, weights, on=['WEEK', 'SCRAM'], how='left')

# Combining weight cols
pulse['HWEIGHT'] = np.where(pulse.HWEIGHT_x.isnull() == False, pulse.HWEIGHT_x, pulse.HWEIGHT_y)

In [11]:
print('\n', color.BOLD + 'Weight Columns' + color.END, '\n')
display(pulse[['HWEIGHT_x', 'HWEIGHT_y', 'HWEIGHT']].head(3), pulse[['HWEIGHT_x', 'HWEIGHT_y', 'HWEIGHT']].tail(3))


 Weight Columns 



,HWEIGHT_x,HWEIGHT_y,HWEIGHT
0,1170.79,NaN,1170.79
1,899.97,NaN,899.97
2,2077.84,NaN,2077.84


,HWEIGHT_x,HWEIGHT_y,HWEIGHT
2167924,NaN,230.53,230.53
2167925,NaN,9.67,9.67
2167926,NaN,80.42,80.42


<hr>

## Creating data categories
### HOUSEHOLDS WITH KIDS

I am interested in identifying households with individuals under 18 and households with school-age children. In the weekly Pulse survey, the Census asks individuals how many people under 18 live in their households. That number is recorded in the <b>THHLD_NUMKID</b> column. This column does not have missing values.

In [12]:
pulse.THHLD_NUMKID.value_counts()

0    1399999
1     333974
2     275718
3     105144
4      35152
5      17940
Name: THHLD_NUMKID, dtype: int64

I create a new <b>kids</b> column where I mark 1 for households with one or more people under 18 and 0 for those without. 

In [13]:
pulse['kids'] = np.where(pulse.THHLD_NUMKID == 0, 0, 1)

I then label households with children in school or homeschooled. The database records answers to the question,

    'At any time during the 2020-2021 school year, were, or will, any children in this household enrolled in a public school, enrolled in a private school, or educated in a homeschool setting in Kindergarten through 12th grade or grade equivalent? Select all that apply.'

in three columns:

- <b>ENROLL1</b>: 'Yes, enrolled in a public or private school'
- <b>ENROLL2</b>: 'Yes, homeschooled'
- <b>ENROLL3</b>: 'No'

These columns include missing responses marked as '-88' in the database. If the interviewee addressed the question but did not select the category in a particular column, their response is marked as '-99'.

In [14]:
print(pulse.ENROLL1.value_counts(), '\n')
print(pulse.ENROLL2.value_counts(), '\n')
print(pulse.ENROLL3.value_counts())

-88    1480863
 1      494499
-99     192565
Name: ENROLL1, dtype: int64 

-88    1480863
-99     649647
 1       37417
Name: ENROLL2, dtype: int64 

-88    1480863
-99     529052
 1      158012
Name: ENROLL3, dtype: int64


I remove rows where households reported having members 18 or younger, but -88 or -99 is marked in all three ENROLL columns.

In [15]:
# removing invalid records
pulse = pulse[~(
    (pulse.kids == 1) & 
    (pulse.ENROLL1 < 0) & 
    (pulse.ENROLL2 < 0) & 
    (pulse.ENROLL3 < 0)
)]

print('\n', color.BOLD + 'Original row count:' + color.END, original_length, 
      '\n', color.BOLD + 'Count of valid records:' + color.END, len(pulse), 
      '\n', color.BOLD + 'Total removed:' + color.END, original_length - len(pulse), '\n')


 Original row count: 2167927 
 Count of valid records: 2045478 
 Total removed: 122449 



In a new column, <b>school_kids</b>, I mark 1 for households which reported having chilren enrolled in school or homeschooled, and 0 for households which answered that they did not. 

In [16]:
# Labeling households with school kids, marking na for others
pulse['school_kids'] = np.where(
    (pulse.kids == 1) &
    (
        (pulse.ENROLL1 == 1) | (pulse.ENROLL2 == 1)
    ), 1, np.nan)

# Labeling households without school kids
pulse['school_kids'] = np.where(
    (pulse.kids == 1) &
    (
        (pulse.ENROLL3 == 1)
    ), 0, pulse.school_kids)


print('\n', color.BOLD + 'Kids and school kids columns' + color.END, '\n')
display(pulse[['kids', 'ENROLL1', 'ENROLL2', 'ENROLL3', 'school_kids']].head(3))


 Kids and school kids columns 



,kids,ENROLL1,ENROLL2,ENROLL3,school_kids
1,0,-88,-88,-88,NaN
2,1,1,-99,-99,1.00
3,0,-88,-88,-88,NaN


### HOUSEHOLDS WHICH EXPERIENCED A RECENT JOB LOSS

Interviewees in the survey were asked if they had experienced a loss of employment income since March 13, 2020. Their answers were recorded in the <b>WRKLOSS</b> column. The column includes missing values. 

In [17]:
pulse.WRKLOSS.value_counts()

 2     1246640
 1      787799
-99      10798
-88        241
Name: WRKLOSS, dtype: int64

Because this question required interviewees to select only one answer, I remove all answers marked as -88 (missing) and -99 (question seen but category not selected). 

In [18]:
pulse = pulse[~((pulse.WRKLOSS == -88) | (pulse.WRKLOSS == -99))]

print('\n', color.BOLD + 'Original row count:' + color.END, original_length, 
      '\n', color.BOLD + 'Count of valid records:' + color.END, len(pulse), 
      '\n', color.BOLD + 'Total removed:' + color.END, original_length - len(pulse))


 Original row count: 2167927 
 Count of valid records: 2034439 
 Total removed: 133488


In [19]:
pulse.WRKLOSS.value_counts()

2    1246640
1     787799
Name: WRKLOSS, dtype: int64

I create a new column, <b>recent_job_loss</b>, where I mark 1 for households who reported a loss (currently marked 1) and 0 for households who didn't (currently marked 2). 

In [20]:
# Labeling households which experienced recent job losses
pulse['recent_job_loss'] = np.where(pulse.WRKLOSS == 1, 1, np.nan)
pulse['recent_job_loss'] = np.where(pulse.WRKLOSS == 2, 0, pulse['recent_job_loss'])

print('\n', color.BOLD + 'Recent job loss columns' + color.END, '\n')
display(pulse[['WRKLOSS', 'recent_job_loss']].head(3))


 Recent job loss columns 



,WRKLOSS,recent_job_loss
1,1,1.00
2,2,0.00
3,1,1.00


### MARITAL STATUS

I follow the same process to recode the marital status of the interviewee. The <b>MS</b> column includes the following choices:

1) Now married 
2) Widowed
3) Divorced
4) Separated
5) Never married

As above, -99 indicates that the question was seen but the category was not selected and -88 indicates that the answer is missing. Because the question required interviewees to select only one answer, I mark as null all answers not ranging from 1 to 5. I do not remove invalid records because this characteristic is not central to my analysis. 

In [21]:
# Labeling interviewees who reported not being married
pulse['married'] = np.where(
    (pulse.MS == 2) | 
    (pulse.MS == 3) | 
    (pulse.MS == 4) | 
    (pulse.MS == 5), 0, np.nan)

# Labeling interviewees who being married
pulse['married'] = np.where(pulse.MS == 1, 1, pulse.married)

print('\n', color.BOLD + 'Marital status columns' + color.END, '\n')
display(pulse[['MS', 'married']].head(3))


 Marital status columns 



,MS,married
1,3,0.00
2,1,1.00
3,2,0.00


### INCOME

I retrieved low-income limits data published in 2020 by the Department of Housing and Urban Development from https://www.huduser.gov/portal/datasets/il/il20/State-Incomelimits-Report-FY20r.pdf. I used SmallPDF to convert the documents into Excel files. I then filtered the data to only include low-income limits and added the <b>EST-ST</b> column with state codes as reported in the Census Pulse Survey data dictionary.

State income limits rely on household member counts. However, limits are only defined for households of up to 8 members. Larger households meet the 8-member limits. To make sure I could match all of the data, I create a new <b>member_count</b> column which reports the exact number of household members as reported in the <b>THHLD_NUMPER</b> survey column for households of 8 members or fewer. I record 8 for larger households. 

In [22]:
# Creating member counts col
pulse['member_count'] = np.where(pulse.THHLD_NUMPER <= 8, pulse.THHLD_NUMPER, 8)

I import the HUD state low-income limits data.

In [23]:
# Importing df
lil = pd.read_csv('State-Incomelimits-Report-FY20r-lil.csv')

print('\n', color.BOLD + 'State Low-Income Limits 2020' + color.END, '\n')
display(lil.head(3))


 State Low-Income Limits 2020 



,state,EST_ST,1,2,3,4,5,6,7,8
0,ALABAMA,1,36550.00,41800.00,47000.00,52250.00,56400.00,60600.00,64800.00,68950.00
1,ALASKA,2,51650.00,59000.00,66400.00,73750.00,79650.00,85550.00,91450.00,97350.00
2,ARIZONA,4,40400.00,46150.00,51900.00,57700.00,62300.00,66900.00,71500.00,76150.00


I transpose the member count columns of the income data to create a more compact dataframe.

In [24]:
# Transposing df
lil_t = lil.melt(id_vars=['state','EST_ST']).rename(columns={
    'variable': 'member_count',
    'value': 'low_income_limit'
})
# Converting member_count col to integer
lil_t.member_count = lil_t.member_count.astype('int64')

print('\n', color.BOLD + 'State Low-Income Limits 2020 - Transposed' + color.END, '\n')
display(lil_t.head(3))


 State Low-Income Limits 2020 - Transposed 



,state,EST_ST,member_count,low_income_limit
0,ALABAMA,1,1,36550.00
1,ALASKA,2,1,51650.00
2,ARIZONA,4,1,40400.00


I then left-merge the pulse dataframe with the income dataframe on the state code column, <b>EST_ST</b>, and household member count column, <b>member_count</b>. As a result, each row is assigned to the appropriate low-income limit for the state and household member count of the household it represents.

In [25]:
pulse = pd.merge(pulse, lil_t, on=['EST_ST', 'member_count'], how='left')      

print('\n', color.BOLD + 'Pulse & State Low-Income Limits data merge' + color.END, '\n')
display(pulse.head(3))


 Pulse & State Low-Income Limits data merge 



,SCRAM,WEEK,EST_ST,EST_MSA,REGION,HWEIGHT_x,PWEIGHT,TBIRTH_YEAR,ABIRTH_YEAR,EGENDER,...,TBEDROOMS,HWEIGHT_y,HWEIGHT,kids,school_kids,recent_job_loss,married,member_count,state,low_income_limit
0,V220000001S10011554410113,22,1,NaN,2.00,899.97,1683.63,1947,2,2,...,NaN,NaN,899.97,0,NaN,1.00,0.00,2,ALABAMA,41800.00
1,V220000001S15010024400123,22,1,NaN,2.00,2077.84,3887.14,1989,2,1,...,NaN,NaN,2077.84,1,1.00,0.00,1.00,6,ALABAMA,60600.00
2,V220000001S15010351400113,22,53,42660.00,4.00,3555.42,6731.73,1971,2,2,...,NaN,NaN,3555.42,0,NaN,1.00,0.00,2,WASHINGTON,57450.00


The Census Pulse data does not provide exact household incomes. Instead, it provides income brackets labeled with integers from 1 to 8 as follows:

1) Less than \\$25,000  
2) \\$25,000 - \\$34,999  
3) \\$35,000 - \\$49,999   
4) \\$50,000 - \\$74,999   
5) \\$75,000 - \\$99,999   
6) \\$100,000 - \\$149,999   
7) \\$150,000 - \\$199,999
8) \\$200,000 and above

I use the same brackets to categorize low-income limits in a new column, <b>low_income_limit_cat</b>.

In [26]:
# Creating new income cat col
pulse['low_income_limit_cat'] = np.where(pulse.low_income_limit < 25000, 1, np.nan)

pulse['low_income_limit_cat'] = np.where(
    (pulse.low_income_limit >= 25000) & 
    (pulse.low_income_limit <= 34999), 2, pulse.low_income_limit_cat)

pulse['low_income_limit_cat'] = np.where(
    (pulse.low_income_limit >= 35000) & 
    (pulse.low_income_limit <= 49999), 3, pulse.low_income_limit_cat)

pulse['low_income_limit_cat'] = np.where(
    (pulse.low_income_limit >= 50000) & 
    (pulse.low_income_limit <= 74999), 4, pulse.low_income_limit_cat)

pulse['low_income_limit_cat'] = np.where(
    (pulse.low_income_limit >= 75000) & 
    (pulse.low_income_limit <= 99999), 5, pulse.low_income_limit_cat)

pulse['low_income_limit_cat'] = np.where(
    (pulse.low_income_limit >= 100000) & 
    (pulse.low_income_limit <= 149999), 6, pulse.low_income_limit_cat)

I specify conditions only for the first categories because low income limits do not top $149,999 across all states.

In [27]:
print('\n', color.BOLD + 'Low-income limit categories:' + color.END, pulse.low_income_limit_cat.sort_values().unique(), '\n')


 Low-income limit categories: [2. 3. 4. 5. 6.] 



Finally, I compare the income categories reported by each household in the survey to the low-income limit category which corresponds to that household. 

In [28]:
pulse['low_income'] = np.where(pulse.INCOME <= pulse.low_income_limit_cat, 1, 0)

print('\n', color.BOLD + 'Low-income limit comparison columns' + color.END, '\n')
display(pulse[['INCOME', 'low_income_limit', 'low_income_limit_cat', 'low_income']].tail(5))


 Low-income limit comparison columns 



,INCOME,low_income_limit,low_income_limit_cat,low_income
2034434,3,57400.00,4.00,1
2034435,4,44650.00,3.00,0
2034436,4,63750.00,4.00,1
2034437,4,44650.00,3.00,0
2034438,5,51000.00,4.00,0


<hr>

## Analyzing the data

First, I calculate a weighted count of job losses. I create a new column, <b>recent_job_loss_weight</b> where I multiply the weight of the column <b>HWEIGHT</b> and the <b>recent_job_loss</b> value (1 if 'Yes' and 0 if 'No'). 

In [29]:
# Creating weighted job loss col
pulse['recent_job_loss_weight'] = pulse.recent_job_loss * pulse.HWEIGHT

### USEFUL FUNCTIONS

Before I analyze the data, I create a couple of useful functions which will make the process more efficient. 

The <b>group_data</b> function groups a dataframe and calculates for each group a sum of all households and a sum of households which experienced recent job losses. Then, the function calculates the percentage of households in each group which reported recent job loss by dividing the weighted sum of households reporting losses by the sum of all households in the group. 

The <b>name_group</b> functions each create a new column and assign group names according to each household's characteristics. 

The <b>pivot_data</b> function takes the grouped data, sets a list of given columns as index and pivots a list of given columns, setting the calculated percentages of recent job loss as values. 

In [30]:
def group_data(df, groups):
    grouped_df = df.groupby(by=groups, as_index=False, dropna=False).agg(
    {'recent_job_loss_weight': 'sum',
    'HWEIGHT': 'sum'})
    grouped_df['pct_recent_job_loss'] = grouped_df.recent_job_loss_weight / grouped_df.HWEIGHT * 100
    return grouped_df

In [31]:
def name_group_kids(grouped_df):
    grouped_df['group_kids'] = np.where(grouped_df.kids == 0, 'Households without children', np.nan)

    grouped_df['group_kids'] = np.where(
        (grouped_df.kids == 1) &
        (grouped_df.school_kids == 0), 'Households with children not in school', grouped_df.group_kids)

    grouped_df['group_kids'] = np.where(
        (grouped_df.kids == 1) &
        (grouped_df.school_kids == 1), 'Households with children in school', grouped_df.group_kids)
    
    return grouped_df

In [32]:
def name_group_married(grouped_df):
    grouped_df['group_married'] = np.where(grouped_df.married == 1, 'Married', np.nan)
    grouped_df['group_married'] = np.where(grouped_df.married == 0, 'Single', grouped_df.group_married)
    return grouped_df  

In [33]:
def name_group_income(grouped_df):
    grouped_df['group_income'] = np.where(grouped_df.low_income == 1, 'Low income', np.nan)
    grouped_df['group_income'] = np.where(grouped_df.low_income == 0, 'Middle or upper income', grouped_df.group_income)
    return grouped_df

In [34]:
def pivot_data(grouped_df, index_ls, columns_ls):
    pivoted_df = grouped_df.pivot(index=index_ls, columns=columns_ls, values='pct_recent_job_loss').reset_index()
    return pivoted_df

### STATE ANALYSIS

#### What percentage of households with and without children have experienced job losses since the beginning of the pandemic in each state (on the average week)?

I group the dataframe by the <b>state</b>, <b>kids</b> and <b>school_kids</b> columns.

In [35]:
# Grouping data by state
state_grouped = group_data(pulse, ['state', 'WEEK', 'kids', 'school_kids'])

print('\n', color.BOLD + 'Recent job loss by state and week' + color.END, '\n')
display(state_grouped.head(3))


 Recent job loss by state and week 



,state,WEEK,kids,school_kids,recent_job_loss_weight,HWEIGHT,pct_recent_job_loss
0,ALABAMA,1,0,NaN,357294.52,1096802.00,32.58
1,ALABAMA,1,1,0.00,48913.60,126276.89,38.74
2,ALABAMA,1,1,1.00,263651.51,538063.06,49.00


In [36]:
# Grouping data by state and week, calculating avg
state_grouped_avg = state_grouped.groupby(by=['state', 'kids', 'school_kids'])['pct_recent_job_loss'].mean().reset_index()

print('\n', color.BOLD + 'Recent job loss by state on the avg week' + color.END, '\n')
display(state_grouped_avg.head(4))


 Recent job loss by state on the avg week 



,state,kids,school_kids,pct_recent_job_loss
0,ALABAMA,1,0.00,41.01
1,ALABAMA,1,1.00,49.13
2,ALASKA,1,0.00,45.05
3,ALASKA,1,1.00,49.29


I use the naming function to assign appropriate names to each group in a new <b>group_kids</b> column.

In [37]:
# Naming groups
state_grouped_avg = name_group_kids(state_grouped_avg)

print('\n', color.BOLD + 'Recent job loss by state on the avg week (with group names)' + color.END, '\n')
display(state_grouped_avg.head(3))


 Recent job loss by state on the avg week (with group names) 



,state,kids,school_kids,pct_recent_job_loss,group_kids
0,ALABAMA,1,0.00,41.01,Households with children not in school
1,ALABAMA,1,1.00,49.13,Households with children in school
2,ALASKA,1,0.00,45.05,Households with children not in school


I then pivot the data to display it in a more compact format. I <b>sort</b> the table in descending order by job losses experienced by families with school-age children.

In [38]:
# Pivoting df
state_pivot = pivot_data(state_grouped_avg, 'state', 'group_kids').sort_values(by='Households with children in school', ascending=False)

print('\n', color.BOLD + 'Recent job loss by state on the avg week (pivoted)' + color.END, '\n')
display(state_pivot.head(3))


 Recent job loss by state on the avg week (pivoted) 



group_kids,state,Households with children in school,Households with children not in school
28,NEVADA,64.25,57.71
22,MICHIGAN,61.97,50.65
4,CALIFORNIA,59.12,51.68


### CALIFORNIA WEEKLY ANALYSIS

#### What percentage of households with and without school-age children / married / low income have experienced job losses since the beginning of the pandemic in California? 

First, I create a new subset of California households.

In [39]:
# Filtering California households
pulse_cal = pulse[pulse.state == 'CALIFORNIA']

print('\n', color.BOLD + 'California data' + color.END, '\n')
display(pulse_cal.head(3))


 California data 



,SCRAM,WEEK,EST_ST,EST_MSA,REGION,HWEIGHT_x,PWEIGHT,TBIRTH_YEAR,ABIRTH_YEAR,EGENDER,...,kids,school_kids,recent_job_loss,married,member_count,state,low_income_limit,low_income_limit_cat,low_income,recent_job_loss_weight
3,V220000001S18010504900113,22,6,NaN,4.00,1187.56,2190.21,1985,2,2,...,0,NaN,0.00,1.00,2,CALIFORNIA,55750.00,4.00,0,0.00
31,V220000005S12051860410113,22,6,31080.00,4.00,1518.05,2799.71,1956,2,2,...,0,NaN,0.00,1.00,2,CALIFORNIA,55750.00,4.00,0,0.00
38,V220000006S16060972200113,22,6,NaN,4.00,5012.50,9244.46,1979,2,2,...,1,1.00,0.00,1.00,5,CALIFORNIA,75250.00,5.00,1,0.00


I then use the predefined functions to group, name and pivot two new subsets of the data: <b>cal_grouped_married</b> and <b>cal_grouped_low_income</b>. Each dataframe categorizes households by the marital status of the interviewee and the income bracket of the household respectively. 

In [40]:
cal_grouped_married = group_data(pulse_cal, ['WEEK', 'kids', 'school_kids', 'married'])
cal_grouped_income = group_data(pulse_cal, ['WEEK', 'kids', 'school_kids', 'low_income'])

print('\n', color.BOLD + 'Recent job loss in California by week - Marital Status' + color.END, '\n')
display(cal_grouped_married.head(3))

print('\n', color.BOLD + 'Recent job loss in California by week - Income Bracket' + color.END, '\n')
display(cal_grouped_income.head(3))


 Recent job loss in California by week - Marital Status 



,WEEK,kids,school_kids,married,recent_job_loss_weight,HWEIGHT,pct_recent_job_loss
0,1,0,NaN,0.00,1963522.39,4338013.63,45.26
1,1,0,NaN,1.00,1388569.52,3342478.49,41.54
2,1,0,NaN,NaN,11715.97,31271.07,37.47



 Recent job loss in California by week - Income Bracket 



,WEEK,kids,school_kids,low_income,recent_job_loss_weight,HWEIGHT,pct_recent_job_loss
0,1,0,NaN,0,1344270.02,3422803.15,39.27
1,1,0,NaN,1,2019537.87,4288960.04,47.09
2,1,1,0.00,0,117648.41,371833.77,31.64


In [41]:
# Naming groups
cal_grouped_married = name_group_kids(cal_grouped_married)
cal_grouped_income = name_group_kids(cal_grouped_income)

cal_grouped_married = name_group_married(cal_grouped_married)
cal_grouped_income = name_group_income(cal_grouped_income)

print('\n', color.BOLD + 'Recent job loss in California by week (with group names) - Marital Status' + color.END, '\n')
display(cal_grouped_married.head(3))

print('\n', color.BOLD + 'Recent job loss in California by week (with group names) - Income Bracket' + color.END, '\n')
display(cal_grouped_income.head(3))


 Recent job loss in California by week (with group names) - Marital Status 



,WEEK,kids,school_kids,married,recent_job_loss_weight,HWEIGHT,pct_recent_job_loss,group_kids,group_married
0,1,0,NaN,0.00,1963522.39,4338013.63,45.26,Households without children,Single
1,1,0,NaN,1.00,1388569.52,3342478.49,41.54,Households without children,Married
2,1,0,NaN,NaN,11715.97,31271.07,37.47,Households without children,nan



 Recent job loss in California by week (with group names) - Income Bracket 



,WEEK,kids,school_kids,low_income,recent_job_loss_weight,HWEIGHT,pct_recent_job_loss,group_kids,group_income
0,1,0,NaN,0,1344270.02,3422803.15,39.27,Households without children,Middle or upper income
1,1,0,NaN,1,2019537.87,4288960.04,47.09,Households without children,Low income
2,1,1,0.00,0,117648.41,371833.77,31.64,Households with children not in school,Middle or upper income


I remove rows with null group values from the dataframe grouped on marital status.

In [42]:
# Removing nulls from col
cal_grouped_married = cal_grouped_married[cal_grouped_married.group_married != 'nan']

Finally, I pivot both dataframes. 

In [43]:
# Pivoting df
married_pivot = pivot_data(cal_grouped_married, 'WEEK', ['group_kids', 'group_married'])
income_pivot = pivot_data(cal_grouped_income, 'WEEK', ['group_kids', 'group_income'])

print('\n', color.BOLD + 'Recent job loss by week (pivoted) - Marital Status' + color.END, '\n')
display(married_pivot.head(5))

print('\n', color.BOLD + 'Recent job loss by week (pivoted) - Income Bracket' + color.END, '\n')
display(income_pivot.head(5))


 Recent job loss by week (pivoted) - Marital Status 



group_kids    WEEK Households without children          \
group_married                           Single Married   
0                1                       45.26   41.54   
1                2                       40.86   43.08   
2                3                       47.99   43.76   
3                4                       46.66   37.36   
4                5                       44.26   37.56   

group_kids    Households with children not in school          \
group_married                                 Single Married   
0                                              57.70   48.42   
1                                              76.94   46.58   
2                                              67.83   61.01   
3                                              69.01   66.05   
4                                              72.28   41.09   

group_kids    Households with children in school          
group_married                             Single Married  
0                                          64.90   51.11  
1                                          66.81   61.60  
2                                          67.15   55.00  
3                                          69.65   60.45  
4                                          66.29   53.56


 Recent job loss by week (pivoted) - Income Bracket 



group_kids   WEEK Households without children             \
group_income           Middle or upper income Low income   
0               1                       39.27      47.09   
1               2                       38.51      43.92   
2               3                       41.32      48.87   
3               4                       34.25      49.94   
4               5                       33.54      47.71   

group_kids   Households with children not in school             \
group_income                 Middle or upper income Low income   
0                                             31.64      64.80   
1                                             18.83      80.39   
2                                             29.13      80.68   
3                                             41.16      82.32   
4                                             31.51      70.19   

group_kids   Households with children in school             
group_income             Middle or upper income Low income  
0                                         41.19      66.16  
1                                         50.74      71.18  
2                                         39.92      72.81  
3                                         44.42      76.85  
4                                         41.56      68.59

### CALIFORNIA METRO WEEKLY ANALYSIS

#### Where did California households with and without school-age children experience job losses? 

The Pulse Survey includes households from three California metropolitan areas:

- Los Angeles - Long Beach - Anaheim
- San Francisco - Oakland - Hayward
- Riverside - San Bernardino - Ontario

These regions are coded with numeric entries in the <b>EST_MSA</b> column. I import a csv file which pairs each code to the appropriate metropolitan area name. I then merge the file with my dataframe. 

In [44]:
# Importing df
metro_codes = pd.read_csv('metro_codes.csv')

# Merging with cal data
pulse_cal = pd.merge(pulse_cal, metro_codes, on='EST_MSA', how='left')

I group, name and pivot the data. I remove rows with null metropolitan area entries.

In [45]:
# Grouping df
cal_grouped_metro = group_data(pulse_cal, ['WEEK', 'kids', 'school_kids', 'metro_name'])

# Removing nulls
cal_grouped_metro = cal_grouped_metro[cal_grouped_metro.metro_name.isnull() == False]

print('\n', color.BOLD + 'Recent job loss in California by week - Metro Area' + color.END, '\n')
display(cal_grouped_metro.head(3))


 Recent job loss in California by week - Metro Area 



,WEEK,kids,school_kids,metro_name,recent_job_loss_weight,HWEIGHT,pct_recent_job_loss
0,1,0,NaN,Los Angeles - Long Beach - Anaheim,1465816.05,2802935.13,52.30
1,1,0,NaN,Riverside - San Bernardino - Ontario,358333.78,988361.36,36.26
2,1,0,NaN,San Francisco - Oakland - Hayward,414629.74,1074667.99,38.58


In [46]:
# Naming groups
cal_grouped_metro = name_group_kids(cal_grouped_metro)

print('\n', color.BOLD + 'Recent job loss in California by week (with group names) - Metro Area' + color.END, '\n')
display(cal_grouped_metro.head(3))


 Recent job loss in California by week (with group names) - Metro Area 



,WEEK,kids,school_kids,metro_name,recent_job_loss_weight,HWEIGHT,pct_recent_job_loss,group_kids
0,1,0,NaN,Los Angeles - Long Beach - Anaheim,1465816.05,2802935.13,52.30,Households without children
1,1,0,NaN,Riverside - San Bernardino - Ontario,358333.78,988361.36,36.26,Households without children
2,1,0,NaN,San Francisco - Oakland - Hayward,414629.74,1074667.99,38.58,Households without children


In [47]:
# Pivoting df
metro_pivot = pivot_data(cal_grouped_metro, 'WEEK', ['group_kids', 'metro_name'])

print('\n', color.BOLD + 'Recent job loss by week (pivoted) - Metro Areas' + color.END, '\n')
display(metro_pivot.head(5))


 Recent job loss by week (pivoted) - Metro Areas 



group_kids WEEK        Households without children  \
metro_name      Los Angeles - Long Beach - Anaheim   
0             1                              52.30   
1             2                              47.25   
2             3                              52.62   
3             4                              48.13   
4             5                              46.40   

group_kids                                       \
metro_name Riverside - San Bernardino - Ontario   
0                                         36.26   
1                                         46.81   
2                                         51.77   
3                                         46.53   
4                                         44.59   

group_kids                                    \
metro_name San Francisco - Oakland - Hayward   
0                                      38.58   
1                                      41.08   
2                                      37.84   
3                                      33.61   
4                                      42.28   

group_kids Households with children not in school  \
metro_name     Los Angeles - Long Beach - Anaheim   
0                                           42.75   
1                                           68.73   
2                                           72.95   
3                                           70.05   
4                                           68.32   

group_kids                                       \
metro_name Riverside - San Bernardino - Ontario   
0                                         42.73   
1                                         81.08   
2                                         78.95   
3                                         68.55   
4                                         83.36   

group_kids                                    \
metro_name San Francisco - Oakland - Hayward   
0                                      44.95   
1                                      46.22   
2                                      46.50   
3                                      69.62   
4                                      45.04   

group_kids Households with children in school  \
metro_name Los Angeles - Long Beach - Anaheim   
0                                       58.48   
1                                       54.92   
2                                       59.27   
3                                       70.25   
4                                       60.58   

group_kids                                       \
metro_name Riverside - San Bernardino - Ontario   
0                                         59.81   
1                                         68.96   
2                                         58.83   
3                                         69.01   
4                                         63.88   

group_kids                                    
metro_name San Francisco - Oakland - Hayward  
0                                      54.59  
1                                      56.34  
2                                      49.89  
3                                      47.98  
4                                      44.42

#### How many households with children have experienced job losses since the beginning of the pandemic in California?

Although Census Pulse data can produce accurate percentages and ratios, the Census advises against using the data to calculate raw totals. As a solution, I compare the percentages I calculated to Census statistics from the American Community Survey (ACS). 

There are about 4,360,100 households in California with one or more people under 18, according to 2019 ACS <a href='https://data.census.gov/cedsci/table?q=households%20&g=0400000US06&tid=ACSDP1Y2019.DP02&hidePreview=false'>estimates</a>. 

Because my data is broken down by school enrollment, I calculate the ratio of California households with children enrolled in school to California households without children enrolled in school. 

In [48]:
# Creating grouped chart
cal_kids_stats = pulse_cal[['kids', 'school_kids', 'HWEIGHT']].groupby(by=['kids', 'school_kids'])['HWEIGHT'].sum().reset_index()

# Calculating pct of total
cal_kids_stats['pct_of_total'] = cal_kids_stats.HWEIGHT / cal_kids_stats.HWEIGHT.sum()

print('\n', color.BOLD + 'California Households with children in survey sample' + color.END, '\n')
display(cal_kids_stats)


 California Households with children in survey sample 



,kids,school_kids,HWEIGHT,pct_of_total
0,1,0.00,19779068.94,0.21
1,1,1.00,73958729.37,0.79


I use the ACS estimate to calculate my statistics. First, I calculate the approximate number of California households with children in and out of school. Then, I calculate how many of those households experienced recent job losses. 

In [49]:
cal_kids_inschool = cal_kids_stats.pct_of_total[1]* 4360100
cal_kids_notinschool = cal_kids_stats.pct_of_total[0]* 4360100

cal_kids_inschool_loss = 0.5912 * cal_kids_inschool
cal_kids_notinschool_loss = 0.5168 * cal_kids_notinschool



print('\n', color.BOLD + 'California households with children' + color.END, '\n'
     '\n In school:', round(cal_kids_inschool, 2),
     '\n Not in school:', round(cal_kids_notinschool, 2))

print('\n', color.BOLD + 'California households experiencing job losses with children' + color.END, '\n'
     '\n In school:', round(cal_kids_inschool_loss, 2),
     '\n Not in school:', round(cal_kids_notinschool_loss, 2))


 California households with children 

 In school: 3440100.6 
 Not in school: 919999.4

 California households experiencing job losses with children 

 In school: 2033787.47 
 Not in school: 475455.69


<hr>